In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import load_model

In [2]:
network_loaded = load_model('models/emodet.h5')

In [3]:
network_loaded.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [9]:
face_detector = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')

## Single person image

In [4]:
image = cv2.imread('data/Images/gabriel.png')
cv2.imshow('abc',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
original_image = image.copy()
faces = face_detector.detectMultiScale(original_image)
faces

array([[162,  40, 128, 128]])

In [8]:
roi = image[40:40 + 128, 162:162 + 128]
cv2.imshow('abc',roi)
cv2.waitKey(0)
cv2.destroyAllWindows()
roi = cv2.resize(roi, (48, 48))
roi = roi / 255
roi = np.expand_dims(roi, axis = 0)
probs = network_loaded.predict(roi)
result = np.argmax(probs)
result

1/1 [==============================] - 0s 283ms/step


3

## Multiple faces in image

In [11]:
image = cv2.imread('data/Images/faces_emotions.png')
cv2.imshow('abc',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
faces = face_detector.detectMultiScale(image)

In [14]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [20]:
for (x, y, w, h) in faces:
  cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)
  roi = image[y:y + h, x:x + w]
  roi = cv2.resize(roi, (48, 48))
  roi = roi / 255
  roi = np.expand_dims(roi, axis = 0)
  prediction = network_loaded.predict(roi, verbose=0)
  cv2.putText(image, emotions[np.argmax(prediction)], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2, cv2.LINE_AA)
cv2.imshow('abc',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Videos

In [17]:
cap = cv2.VideoCapture('data/Videos/emotion_test01.mp4')
connected, video = cap.read()
print(connected, video.shape)

True (360, 640, 3)


In [18]:
save_path = 'results/Videos/emotion_test01_result.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24
output_video = cv2.VideoWriter(save_path, fourcc, fps, (video.shape[1], video.shape[0]))

In [21]:
while (cv2.waitKey(1) < 0):
  connected, frame = cap.read()
  if not connected:
    break
  faces = face_detector.detectMultiScale(frame, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))
  if len(faces) > 0:
    for (x, y, w, h) in faces:
      frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
      roi = frame[y:y + h, x:x + w]
      roi = cv2.resize(roi, (48, 48))
      roi = roi / 255
      roi = np.expand_dims(roi, axis = 0)
      prediction = network_loaded.predict(roi, verbose=0)

      if prediction is not None:
        result = np.argmax(prediction)
        cv2.putText(frame, emotions[result], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
  output_video.write(frame)

print('End')
output_video.release()
cv2.destroyAllWindows()

End
